In [ ]:
# The original verison of this code is written by Ole Edvard Grov, 
# Skolelaboratoriet i realfag, UiB

# Importerer et bibliotek, vi skal bruke for å laste ned data.
import requests
import pytz
from datetime import datetime

# Datoen i dag
now=datetime.today().strftime('%Y-%m-%d')
# Navn på filen som skrives ut
outfile='/temp_nedbør_'+now+'.txt'
# Stien til mappen filen skal lages i. # Endre dette til noe på din PC!!!!
sti='W:/Work/Documents/EkteData/EkteData/NAT624'
print('Dataene skrives til filen '+outfile+' i mappen '+sti)
print(' ')


# Setter lokal tidssone, slik at programmet velger rett tid uavhengig av 
# hvilken tidssone maskinen er i.
lokal_tz = pytz.timezone('Europe/Oslo')

# For å hente et varsel trenger vi koodinater lengdegrad og breddegrad. Disse 
# må endres av hver student som skal kjøre koden for et nytt sted.

lengdegrad = 5.3748
breddegrad = 60.36765

params = {
  'lat' : breddegrad,
  'lon' : lengdegrad 
}

# Ignorer denne linja, det forteller bare yr at vi skal behandles som en 
# nettleser.
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) '+
    'Gecko/20100101 Firefox/47.0'
}

# URL vi skal hente data fra.
data_url = 'https://api.met.no/weatherapi/locationforecast/2.0/compact?'

# Henter data fra stedet vi har oppgitt når vi la inn lengde- og breddegrad. 
# Vi sender inn vår informasjon og ber om å få ut data for vårt sted
response = requests.get(data_url, params=params, headers=headers)


# Header. Når vi bruker \t betyr det bare at kolonnene skal skrives ut med en 
# tab mellom.
print('Dato/tid\tNedbør (mm)\tNedbørsum 24 timer (mm)\tLufttemperatur (ºC)')

# Vi behandler data vi får tilbake fra serveren. Responsen 200 betyr at vi har 
# fått et svar. Vi henter ut tidspunkt og setter det til den lokale tiden vi 
# har valgt.
# Linjen som starter med "nedbor" er en måte å hente ut data på fra den 
# strukturen som Met bruker. Den skal fortsette så lenge det finnes timesvarsel
# for nedbør (så lenge nedbør er ulik "None").
if response.status_code == 200:
    data = response.json()['properties']['timeseries']
    if len(data) > 0:
        sum_precip = 0.0
        
        # Denne linjen lager en ny tekst-fil som heter "temp_nedbør" med et
        # datostempel
        # Du kan kommentere ut denne linjen med "#" om du ikke vil lage en fil
        with open(sti+outfile, 'w') as f:
            f.write(
                'Dato/tid\tNedbør (mm)\tNedbørsum 24 timer (mm)'+
                '\tLufttemperatur (ºC)')

            for d in data:
                dato = datetime.strptime(d.get('time'),"%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=pytz.utc).astimezone(lokal_tz)
                nedbor = d.get('data', {}).get('next_1_hours',{}).get('details', {}).get('precipitation_amount')
                temperatur = d.get('data', {}).get('instant', {}).get('details', {}).get('air_temperature')
                if nedbor != None:
                    sum_precip = sum_precip + nedbor

                # Her velger vi at nedbør skal summeres den 7. timen i døgnet. 
                # Kan endres hvis vi vil bruke andre tidspunkt.
                if (dato.hour == 7):
                    print(
                        dato.strftime('%Y-%m-%d %H:%M'), 
                        nedbor, round(sum_precip,1), 
                        temperatur, sep='\t'
                    )
                    # Skriv den samme linjen med data til filen
                    # # Kommenter ut med "#" om du ikke vil lage en fil
                    f.write(
                        dato.strftime('%Y-%m-%d %H:%M')+'\t'+str(nedbor)+
                        '\t'+str(round(sum_precip,1))+'\t'+str(temperatur)+'\n'
                    )
                    sum = 0
                else:
                    print(
                        dato.strftime('%Y-%m-%d %H:%M'), 
                        nedbor, 0.0, temperatur, sep='\t'
                    )        
                    # Skriv den samme linjen med data til filen
                    # # Kommenter ut med "#" om du ikke vil lage en fil
                    f.write(
                        dato.strftime('%Y-%m-%d %H:%M')+'\t'+str(nedbor)+
                        '\t'+'0.0'+'\t'+str(temperatur)+'\n'
                    )
else:
    print('Nettverksfeil!')
    
    
# Hvis du vil endre hvordan tab'ene separeres, kan du f.eks. bytte ut '\t' 
# med ':'

